In [ ]:
#default_exp train_loop

In [9]:
#hide
#export
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.utils.mem import *
from fastai.vision.gan import *
from PIL import Image

import numpy as np

import torch
import torch.nn.functional as F
import torch.nn as nn

from torch.utils.data import DataLoader
from torch.utils.data.dataset import TensorDataset
import pdb

from HiDT.building_blocks import *
from HiDT.adaptive_unet import *
from HiDT.data_bunch import *
from HiDT.discriminator import *

# Training Wrapper

In [413]:
Learner??

In [455]:
#export
class HiDT(Learner):
    "A gan learner to train a HiDT model"
    def __init__(self, data:DataBunch, content_encoder:nn.Module, style_encoder:nn.Module, 
                 decoder:nn.Module, discriminator:nn.Module, gen_loss_func:LossFunction,
                 crit_loss_func:LossFunction, n_crit=None, n_gen=None, switcher:Callback=None, gen_first:bool=False, switch_eval:bool=True,
                 show_img:bool=True, clip:float=None, **learn_kwargs):
        gan = GANModule(content_encoder, style_encoder, decoder, discriminator)
        loss_func = GANLoss(gen_loss_func, crit_loss_func, gan)
        switcher = ifnone(switcher, partial(FixedGANSwitcher, n_crit=n_crit, n_gen=n_gen))
        super().__init__(data=data, model=gan, loss_func=loss_func, callback_fns=[switcher], **learn_kwargs)
        trainer = GANTrainer(self, clip=clip, switch_eval=switch_eval, show_img=show_img)
        self.gan_trainer = trainer
        self.callbacks.append(trainer)

In [485]:
#export
class GANModule(nn.Module):
    "Wrapper around a `generator` and a `critic` to create a GAN."
    def __init__(self, content_encoder=None, style_encoder=None, decoder=None, discriminator=None, gen_mode:bool=True):
        super().__init__()
        self.gen_mode = gen_mode
        self.content_encoder, self.style_encoder, self.decoder, self.discriminator = content_encoder, style_encoder, decoder, discriminator

    def forward(self, orig, orig2, *args):
        if self.gen_mode:
            return self.forward_gen(orig, orig2)
        else:
            return self.forward_disc(orig, orig2)
    
    def forward_gen(self, orig, orig2):
        #put original images through encoders
        orig_cont, orig_hooks = self.content_encoder(orig)
        orig_style = self.style_encoder(orig)
        
        orig2_cont, orig2_hooks = self.content_encoder(orig2)
        orig2_style = self.style_encoder(orig2)
        
        #reconstruction of original images
        orig_recon = self.decoder(orig_cont, orig_style, orig_hooks)
        orig2_recon = self.decoder(orig2_cont, orig2_style, orig2_hooks)
        
        #translated images
        one2two = self.decoder(orig_cont, orig2_style, orig_hooks)
        two2one = self.decoder(orig2_cont, orig_style, orig2_hooks)
        
        #re-encode translated images
        one2two_style = self.style_encoder(one2two)
        two2one_style = self.style_encoder(two2one)
        one2two_cont, one2two_hooks = self.content_encoder(one2two)
        two2one_cont, two2one_hooks = self.content_encoder(two2one)
        
        #re-decode translated images for cycle loss
        cycled_orig = self.decoder(one2two_cont, two2one_style, one2two_hooks)
        cycled_orig2 = self.decoder(two2one_cont, one2two_style, two2one_hooks)
        
        #random style translation
        rand_style = torch.randn(orig.shape[0], 3)
        one_rand = self.decoder(orig_cont, rand_style, orig_hooks)
        two_rand = self.decoder(orig2_cont, rand_style, orig2_hooks)
        
        #encoder random style image
        one_rand_cont, one_rand_hooks = self.content_encoder(one_rand)
        two_rand_cont, two_rand_hooks = self.content_encoder(two_rand)
        one_rand_style = self.style_encoder(one_rand)
        two_rand_style = self.style_encoder(two_rand)
        
        #random image reconstructions
        one_rand_recon = self.decoder(one_rand_cont, one_rand_style, one_rand_hooks)
        two_rand_recon = self.decoder(two_rand_cont, two_rand_style, two_rand_hooks)
        
        #lots of losses
        return [orig_recon, orig2_recon, orig_style, orig2_style, one2two, two2one, \
                one2two_style, two2one_style, one2two_cont, two2one_cont, cycled_orig, \
               cycled_orig2, one_rand_cont, two_rand_cont, one_rand_style, two_rand_style, \
               one_rand_recon, two_rand_recon]
    
    def forward_disc(self, orig, orig2):
        
        #put original images through encoders
        orig_cont, orig_hooks = self.content_encoder(orig)
        orig_style = self.style_encoder(orig)
        
        orig2_cont, orig2_hooks = self.content_encoder(orig2)
        orig2_style = self.style_encoder(orig2)
        
        #translated images
        one2two = self.decoder(orig_cont, orig2_style, orig_hooks)
        two2one = self.decoder(orig2_cont, orig_style, orig2_hooks)
        
        return orig, orig2, one2two, two2one

    def switch(self, gen_mode:bool=None):
        "Put the model in generator mode if `gen_mode`, in critic mode otherwise."
        self.gen_mode = (not self.gen_mode) if gen_mode is None else gen_mode

In [541]:
class GANLoss(GANModule):
    "Loss function wrapper"
    def __init__(self, loss_funcG:Callable, loss_funcC:Callable, model):
        super().__init__()
        self.loss_funcG,self.loss_funcC,self.model = loss_funcG,loss_funcC,model
        self.content_encoder, self.style_encoder, self.decoder, self.discriminator = model.content_encoder, model.style_encoder, model.decoder, model.discriminator

    def forward(self, *args):
        if self.gen_mode:
            return self.generator(*args)
        else:
            return self.discriminator(*args)
        
    def generator(self, output, *args):
        pdb.set_trace()
        return x_a
    
    def mse(val, targ):
        return 0.5 * torch.mean((val-targ)**2)

    def discriminator(self, output, *args):
        orig, orig2, one2two, two2one = output
        
        return mse(orig, torch.ones(orig.shape)) + mse(orig2, torch.ones(orig2.shape)) \
                + mse(one2two, torch.zeros(one2two.shape)) + mse(orig, torch.zeros(two2one.shape))
        
        
        
        

In [542]:
#export
class GANTrainer(LearnerCallback):
    "Handles GAN Training."
    _order=-20
    def __init__(self, learn:Learner, switch_eval:bool=False, clip:float=None, beta:float=0.98, gen_first:bool=False,
                 show_img:bool=True):
        super().__init__(learn)
        self.switch_eval,self.clip,self.beta,self.gen_first,self.show_img = switch_eval,clip,beta,gen_first,show_img
        self.content_encoder, self.style_encoder, self.decoder, self.discriminator = \
            self.model.content_encoder, self.model.style_encoder, self.model.decoder, self.model.discriminator

    def _set_trainable(self):
        #switching eval modes between disc and generator
        if self.gen_mode:
            self.content_encoder.train()
            self.style_encoder.train()
            self.decoder.train()
            self.discriminator.eval()
            requires_grad(self.content_encoder, True)
            requires_grad(self.style_encoder, True)
            requires_grad(self.decoder, True)
            requires_grad(self.discriminator, False)
        else:
            self.content_encoder.eval()
            self.style_encoder.eval()
            self.decoder.eval()
            self.discriminator.train()
            requires_grad(self.content_encoder, True)
            requires_grad(self.style_encoder, True)
            requires_grad(self.decoder, True)
            requires_grad(self.discriminator, True)

    def on_train_begin(self, *args, **kwargs):
        "Create the optimizers for the generator and critic if necessary, initialize smootheners."
        """
        if not getattr(self,'opt_gen',None):
            self.opt_gen = self.opt.new([nn.Sequential(*flatten_model(self.generator))])
        else: self.opt_gen.lr,self.opt_gen.wd = self.opt.lr,self.opt.wd
        if not getattr(self,'opt_critic',None):
            self.opt_critic = self.opt.new([nn.Sequential(*flatten_model(self.critic))])
        else: self.opt_critic.lr,self.opt_critic.wd = self.opt.lr,self.opt.wd
         """  
        self.gen_mode = self.gen_first
        self.switch(self.gen_mode)
        self.closses,self.glosses = [],[]
        #self.smoothenerG,self.smoothenerC = SmoothenValue(self.beta),SmoothenValue(self.beta)
        #self.recorder.add_metric_names(['gen_loss', 'disc_loss'])
        self.imgs,self.titles = [],[]

    def on_train_end(self, *args, **kwargs):
        "Switch in generator mode for showing results."
        self.switch(gen_mode=True)

    def on_batch_begin(self, last_input, last_target, **kwargs):
        "Clamp the weights with `self.clip` if it's not None, return the correct input."
        if self.gen_mode:
            self.last_input = last_input

        test = {'last_input':last_input,'last_target':last_input}
        return test
    
    def on_backward_begin(self, last_loss, last_output, *args, **kwargs):
        #pdb.set_trace()
        "Record `last_loss` in the proper list."
        """
        last_loss = last_loss.detach().cpu()
        if self.gen_mode:
            self.smoothenerG.add_value(last_loss)
            self.glosses.append(self.smoothenerG.smooth)
            self.last_gen = last_output.detach().cpu()
            last_gen_split = torch.split(self.last_gen, 1, 0)
            self.last_critic_preds_ns = self.gan_trainer.critic(last_gen_split[0].cuda(), last_gen_split[1].cuda())
            self.last_critic_preds_s = self.gan_trainer.critic(last_gen_split[2].cuda(), last_gen_split[3].cuda())
        else:
            self.smoothenerC.add_value(last_loss)
            self.closses.append(self.smoothenerC.smooth)
        """

    def on_epoch_begin(self, epoch, *args, **kwargs):
        "Put the critic or the generator back to eval if necessary."
        self.switch(self.gen_mode)

    def on_epoch_end(self, pbar, epoch, last_metrics, *args, **kwargs):
        "Put the various losses in the recorder and show a sample image."
        if not hasattr(self, 'last_gen') or not self.show_img: return
        """
        data = self.learn.data
        inputBPre = torch.unbind(self.last_input[1], dim=0)
        aToA = im.Image(self.last_gen[0]/2+0.5)
        bToB = im.Image(self.last_gen[1]/2+0.5)
        aToB = im.Image(self.last_gen[2]/2+0.5)
        bToA = im.Image(self.last_gen[3]/2+0.5)
        self.imgs.append(aToA)
        self.imgs.append(aToB)
        self.imgs.append(bToB)
        self.imgs.append(bToA)
        self.titles.append(f'Epoch {epoch}-A to A')
        self.titles.append(f'Epoch {epoch}-A to B')
        self.titles.append(f'Epoch {epoch}-B to B')
        self.titles.append(f'Epoch {epoch}-B to A')
        pbar.show_imgs(self.imgs, self.titles)
        """
        pdb.set_trace()
        return add_metrics(last_metrics, [getattr(self.smoothenerG,'smooth',None),getattr(self.smoothenerC,'smooth',None)])

        
    def switch(self, gen_mode:bool=None, *args):
        "Switch the model, if `gen_mode` is provided, in the desired mode."
        self.gen_mode = (not self.gen_mode) if gen_mode is None else gen_mode
        #self.opt.opt = self.opt_gen.opt if self.gen_mode else self.opt_critic.opt
        self._set_trainable()
        self.model.switch(gen_mode)
        self.loss_func.switch(gen_mode)

In [543]:
#export
class FixedGANSwitcher(LearnerCallback):
    "Switcher to do `n_crit` iterations of the critic then `n_gen` iterations of the generator."
    def __init__(self, learn:Learner, n_crit=5, n_gen=1):
        super().__init__(learn)
        self.n_crit,self.n_gen = 1,1

    def on_train_begin(self, **kwargs):
        "Initiate the iteration counts."
        self.n_c,self.n_g = 0,0

    def on_batch_end(self, iteration, **kwargs):
        "Switch the model if necessary."
        if self.learn.gan_trainer.gen_mode:
            self.n_g += 1
            n_iter,n_in,n_out = self.n_gen,self.n_c,self.n_g
        else:
            self.n_c += 1
            n_iter,n_in,n_out = self.n_crit,self.n_g,self.n_c
        target = n_iter if isinstance(n_iter, int) else n_iter(n_in)
        if target == n_out:
            self.learn.gan_trainer.switch()
            self.n_c,self.n_g = 0,0

In [544]:
data = DoubleImageList.from_folders(Path()/'horse2zebra', 'trainA', 'trainB').split_by_rand_pct(0.2).label_from_folder()
data = ImageDataBunch.create_from_ll(data, bs=1, size=224)

In [545]:
cen = ContentEncoder()
sen = StyleEncoder()
dec = Decoder()
disc = UnconditionalDiscriminator()
l = nn.CrossEntropyLoss()

In [546]:
learn = HiDT(data, cen, sen, dec, disc, l, l)

In [547]:
learn.fit(1)

epoch,train_loss,valid_loss,time


> <ipython-input-541-68de911d53f8>(16)generator()
-> return x_a
(Pdb) len(output)
18
(Pdb) exit


BdbQuit: 

In [ ]:
test= next(iter(data.train_dl))[0][0]

In [447]:
learn.loss_func(test, test)

> <ipython-input-440-006217fa421c>(9)forward()
-> for arg in args:
(Pdb) test = args[0]
(Pdb) test
<function test at 0x1397af598>
(Pdb) one = args[0]
(Pdb) one
<function test at 0x1397af598>
(Pdb) one()
(Pdb) args[0]
self = GANLoss(
  (loss_funcG): CrossEntropyLoss()
  (loss_funcC): CrossEntropyLoss()
  (model): GANModule(
    (content_encoder): ContentEncoder(
      (model): Sequential(
        (0): ResBlock(
          (model): Sequential(
            (0): Conv2dBlock(
              (pad): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
              (norm): InstanceNorm2d(3, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
              (activation): ReLU(inplace=True)
              (conv): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
            )
            (1): Conv2dBlock(
              (pad): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
              (norm): InstanceNorm2d(8, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
              (activatio

(Pdb) exit


BdbQuit: 

In [375]:
test

tensor([[[[-0.6392, -0.8222, -0.8779,  ..., -0.7649, -0.8154, -0.8510],
          [-0.3648, -0.6992, -0.8493,  ..., -0.7087, -0.8902, -0.8764],
          [-0.6777, -0.7222, -0.7463,  ..., -0.4977, -0.7095, -0.8590],
          ...,
          [ 0.4566,  0.4284,  0.6802,  ...,  0.2307,  0.1447,  0.0369],
          [ 0.2565,  0.3287,  0.4526,  ...,  0.2154,  0.1408,  0.1453],
          [ 0.1294,  0.2971,  0.5943,  ...,  0.1459,  0.1148,  0.3333]],

         [[-0.7176, -0.8244, -0.8636,  ..., -0.6283, -0.6774, -0.7333],
          [-0.5042, -0.7525, -0.8647,  ..., -0.7255, -0.8602, -0.8502],
          [-0.6941, -0.7807, -0.8078,  ..., -0.5054, -0.8437, -0.8950],
          ...,
          [ 0.2229,  0.1581,  0.3180,  ...,  0.0757, -0.0556, -0.2046],
          [ 0.0401,  0.0698,  0.1875,  ..., -0.0126, -0.1090, -0.0403],
          [-0.0667,  0.0553,  0.3526,  ..., -0.1286, -0.0962,  0.2392]],

         [[-0.7882, -0.8797, -0.8607,  ..., -0.5750, -0.6552, -0.7569],
          [-0.6358, -0.8129, -

In [438]:
def test(*args):
    for arg in args:
        pdb.set_trace()

In [439]:
test(2)

> <ipython-input-438-c7bd83fd5eba>(2)test()
-> for arg in args:
(Pdb) arg
2
(Pdb) exit


BdbQuit: 